# Setup

In [1]:
%matplotlib ipympl
import matplotlib, matplotlib.pyplot

matplotlib.pyplot.ioff()

from package.apps import Apps

# Queries

## Stateless apps in prod

In [2]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod"
        and app.application_metadata[".onEntry.show"] not in [None, "select-instance", "new-instance"]
    ).select({"On entry": lambda app: (app.application_metadata[".onEntry.show"])}).order_by(
        lambda app: (app["On entry"],)
    ).table()

Env,Org,App,On entry
prod,udir,pbu-prod-demo,"""demo-survey"""
prod,udir,pbu-prod-anonym,"""front-page"""
prod,brg,anonym-oppstartsveilederen,"""stateless"""
prod,brg,lpid-wallet-2024,"""stateless"""
prod,brg,veil-persontransport-anonym,"""stateless"""
prod,brg,veil-servering-anonym,"""stateless"""
prod,brg,veil-servering-anonym-en,"""stateless"""
prod,brg,veileder-godstransport-anonym,"""stateless"""
prod,dmf,drift-rapport,"""stateless"""
prod,brg,oppstartsveileder-as-anonym,"""stateless"""


Count: 14


## Stateless anonymous apps in prod

In [3]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod"
        and app.application_metadata[".onEntry.show"] not in [None, "select-instance", "new-instance"]
        and app.application_metadata[".dataTypes.[].appLogic.allowAnonymousOnStateless", :].some(
            lambda value: value == True
        )
    ).select(
        {
            "On entry": lambda app: app.application_metadata[".onEntry.show"],
            "Anonymous dataTypes": lambda app: app.application_metadata[".dataTypes.[]", :]
            .filter(lambda dataType: dataType[".appLogic.allowAnonymousOnStateless"] == True)
            .map(lambda dataType: dataType[".id"]),
        }
    ).table()

Env,Org,App,On entry,Anonymous dataTypes
prod,brg,anonym-oppstartsveilederen,"""stateless""","""oppstart"""
prod,brg,veil-persontransport-anonym,"""stateless""","""oppstart"""
prod,brg,veil-servering-anonym,"""stateless""","""oppstart"""
prod,brg,veil-servering-anonym-en,"""stateless""","""oppstart"""
prod,brg,veileder-godstransport-anonym,"""stateless""","""oppstart"""
prod,udir,pbu-prod-demo,"""demo-survey""","""survey"""
prod,udir,pbu-prod-kodebruker,"""stateless-survey""","""survey"""
prod,brg,oppstartsveileder-as-anonym,"""stateless""","""oppstart"""


Count: 8
